<a href="https://colab.research.google.com/github/pddmadushan/2024-Hackathon-Team-Ai-Teco/blob/main/Code-Generation-LLM-Local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Install Required Libraries
!pip install -q torch transformers sentence-transformers faiss-cpu accelerate pyngrok flask_cors

# Step 2: Load Required Libraries
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Step 3: Load and Parse RAG Text File
file_path = '/content/SampleTestCode2.txt'  # Change this to your actual file path

with open(file_path, 'r') as f:
    text_data = f.read()

# Step 4: Parsing Logic to Extract Chunks Enclosed in `### ... ###`
import re

# Pattern to match any text enclosed by `### ... ###`
pattern = r'###(.*?)###'

# Extract all matches for text enclosed by ###
matches = re.findall(pattern, text_data, re.DOTALL)

# Check if matches are detected and display them for debugging
if not matches:
    print("No matches found. Please check the file format.")
else:
    print(f"Total matches found: {len(matches)}")

# Format chunks by stripping any extra whitespace
formatted_chunks = [match.strip() for match in matches]

# Final check on chunks
print(f"Number of formatted chunks: {len(formatted_chunks)}")
if formatted_chunks:
    print(f"First formatted chunk:\n{formatted_chunks[0]}")

# Step 5: Load Sentence Transformer Model for Embeddings
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 6: Create Embeddings for Each Chunk
embeddings = embedding_model.encode(formatted_chunks)

# Check the shape of embeddings
print(f"Embeddings shape: {embeddings.shape}")

# Ensure embeddings are valid before proceeding
if embeddings.size == 0:
    raise ValueError("No embeddings were created. Please check your input data.")

# Step 7: Create a FAISS Index for Efficient Retrieval
dimension = embeddings.shape[1]  # Get the number of dimensions for FAISS index
index = faiss.IndexFlatL2(dimension)  # L2 distance index
index.add(np.array(embeddings))  # Add your embeddings to the index

print("FAISS index created and embeddings added successfully.")

# Step 8: Define Retrieval Function with Deduplication
def retrieve_code(query):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), k=1)  # Retrieve only the top 1 similar chunk

    # Get the most relevant chunk based on the top index
    top_chunk = formatted_chunks[indices[0][0]] if indices[0][0] < len(formatted_chunks) else None

    # Clean up any unexpected whitespace or newlines
    return top_chunk.strip() if top_chunk else None

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Total matches found: 1
Number of formatted chunks: 1
First formatted chunk:
Prompt: Create a purchase order from sales order screen. 
namespace TestsInternal.Tests { // Define the test case [TestDescription("PurchaseOrderTC")] public class PurchaseTC : Check { // Create instances of required classes CreatePurchaseOrders CreatePurchaseOrders = new CreatePurchaseOrders(); OrderPo OrderPo = new OrderPo(); public override void Execute() { // Log in to the destination site PxLogin.LoginToDestinationSite(); #region Testcase1: Create a Purchase Order using (TestExecution.CreateTestCaseGroup("Testcase1: Create a purchase order")) { #region Teststep1: Create a Purchase Order using (TestExecution.CreateTestStepGroup("Teststep1: Create a purchase order")) { // Navigate to the appropriate row and perform actions CreatePurchaseOrders.Details.SelectRow(CreatePurchaseOrders.Details.Columns.InventoryID, "DRAGONFR"); CreatePurchaseOrders.Details.Row.VendorID.Select("ALLFRUITS"); // Select Vendor ID Cre

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Embeddings shape: (1, 384)
FAISS index created and embeddings added successfully.


In [2]:
from transformers import LlamaForCausalLM, LlamaTokenizer

model_name = "HuggingFaceH4/zephyr-7b-beta"  # Change this to your desired LLaMA 2 variant
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name, torch_dtype='auto', device_map='auto')

You are using a model of type mistral to instantiate a model of type llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
import torch

def generate_code(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_length=1000,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7,
        )

    generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_code

In [4]:
# Import necessary modules
from flask import Flask, request, jsonify  # Flask for web server functionality
from pyngrok import ngrok  # pyngrok for creating secure tunnels to local servers
import requests  # requests for making HTTP requests
from flask_cors import CORS  # Import CORS for Cross-Origin Resource Sharing
import cv2  # OpenCV for image processing
import numpy as np  # NumPy for numerical computing
import tensorflow as tf  # TensorFlow for deep learning

In [5]:
port_no = 5000 #Defining Port

In [6]:
# Initialize Flask app
app = Flask(__name__)

# Enable CORS for all routes
CORS(app)

# Set ngrok authentication token
ngrok.set_auth_token("2nwGagz0j2vNY2LR06XQUiWh1WK_5LD2WrjNPjBQ5d5E8nxBQ")

# Connect to ngrok and get public URL
port_no = 5000  # Set the port number
public_url = ngrok.connect(port_no).public_url

# Define route for home page
@app.route("/")
def home():
    return f"Running Flask on Google Colab!"  # Return a message indicating Flask is running

# Print the public URL
print(f"To access the Global link please click {public_url}")

To access the Global link please click https://949c-34-125-116-2.ngrok-free.app


In [ ]:
# Define route for generating responses
@app.route('/generate_response', methods=['POST'])
def generate_response():
    # Get prompt from the request
    data = request.json
    userQuery = data.get('userQuery', '')

    print(userQuery)

    # Wrap the prompt using the right chat template
    relevant_chunks = retrieve_code(userQuery)
    context = relevant_chunks
    prompt = f"Based on the following code snippets: {context}, please only  the generate code for and dont add comments as well : {userQuery}"

    # Trim the response, remove instruction manually
    response = generate_code(prompt)

    # Replace actual newlines in the response string for JSON
    response = response.replace('\n', '$')

    print(response)

    # Return the generated response as JSON
    return jsonify({"response": response})

# Start the Flask app
if __name__ == '__main__':
    app.run(port=port_no)  # Run the app on the specified port


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


create a sales order


INFO:werkzeug:127.0.0.1 - - [25/Oct/2024 18:25:50] "POST /generate_response HTTP/1.1" 200 -


Based on the following code snippets: Prompt: Create a purchase order from sales order screen. $namespace TestsInternal.Tests { // Define the test case [TestDescription("PurchaseOrderTC")] public class PurchaseTC : Check { // Create instances of required classes CreatePurchaseOrders CreatePurchaseOrders = new CreatePurchaseOrders(); OrderPo OrderPo = new OrderPo(); public override void Execute() { // Log in to the destination site PxLogin.LoginToDestinationSite(); #region Testcase1: Create a Purchase Order using (TestExecution.CreateTestCaseGroup("Testcase1: Create a purchase order")) { #region Teststep1: Create a Purchase Order using (TestExecution.CreateTestStepGroup("Teststep1: Create a purchase order")) { // Navigate to the appropriate row and perform actions CreatePurchaseOrders.Details.SelectRow(CreatePurchaseOrders.Details.Columns.InventoryID, "DRAGONFR"); CreatePurchaseOrders.Details.Row.VendorID.Select("ALLFRUITS"); // Select Vendor ID CreatePurchaseOrders.Details.Row.POSiteID

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


create sales order


INFO:werkzeug:127.0.0.1 - - [25/Oct/2024 18:30:33] "POST /generate_response HTTP/1.1" 200 -


Based on the following code snippets: Prompt: Create a purchase order from sales order screen. $namespace TestsInternal.Tests { // Define the test case [TestDescription("PurchaseOrderTC")] public class PurchaseTC : Check { // Create instances of required classes CreatePurchaseOrders CreatePurchaseOrders = new CreatePurchaseOrders(); OrderPo OrderPo = new OrderPo(); public override void Execute() { // Log in to the destination site PxLogin.LoginToDestinationSite(); #region Testcase1: Create a Purchase Order using (TestExecution.CreateTestCaseGroup("Testcase1: Create a purchase order")) { #region Teststep1: Create a Purchase Order using (TestExecution.CreateTestStepGroup("Teststep1: Create a purchase order")) { // Navigate to the appropriate row and perform actions CreatePurchaseOrders.Details.SelectRow(CreatePurchaseOrders.Details.Columns.InventoryID, "DRAGONFR"); CreatePurchaseOrders.Details.Row.VendorID.Select("ALLFRUITS"); // Select Vendor ID CreatePurchaseOrders.Details.Row.POSiteID